In [18]:
import re
from pathlib import Path

import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

from P07_af_all.T00_lib.classes_ml import MyUtil
from P07_af_all.T00_lib.utils import check_jupyter

In [19]:
SAVE_DATA = False
MAKE_ONE_HOT = False

In [20]:
if check_jupyter():
    BASE_DIR = Path.cwd()  # Current directory of the running file
    DATA_DIR = BASE_DIR.parent / "T02_combine_features"
    CURRENT_DIR = BASE_DIR
else:
    BASE_DIR = Path.cwd()  # Base directory of the project
    DATA_DIR = BASE_DIR / "src/P07_af_all/T02_combine_features"
    CURRENT_DIR = Path(__file__).resolve().parent

dt = MyUtil.get_dt()
print(f"Current Directory: {CURRENT_DIR}")
print(f"Current Date and Time: {dt}")

# Load data
df = pd.read_excel(DATA_DIR / "S02_data_combined_loc.xlsx")
print(df.shape)
df

Code is running in a Jupyter environment.
Current Directory: c:\Users\admin\Coding\research\weld-ml\src\P07_af_all\T10_feature_importances
Current Date and Time: 2026-01-26_12-03
(378, 135)


,sample_no,R,W,D,position,location,Fx_location,Fy_location,Fz_location,Mz_location,...,"Mz__weld__fft_coefficient__attr_""angle""__coeff_16","Mz__weld__fft_coefficient__attr_""real""__coeff_15","Mz__weld__fft_coefficient__attr_""abs""__coeff_58","Mz__weld__fft_coefficient__attr_""abs""__coeff_83","Mz__weld__fft_coefficient__attr_""abs""__coeff_96","Mz__weld__fft_coefficient__attr_""angle""__coeff_37","Mz__weld__fft_coefficient__attr_""imag""__coeff_2",stress_value_5052,stress_value_6061,stress_value_center
0,1,1400,60,10,0.153846,1,-0.077671,0.143026,1.244326,1.873865,...,48.966568,-50.258868,770.788243,599.048302,791.064441,-15.139956,6594.092485,28.0,51.0,12.0
1,2,1400,60,15,0.153846,1,-0.133276,0.164254,1.203367,-1.054677,...,-4.784945,1601.554004,358.604392,111.493021,204.891839,-83.550988,6665.135623,14.0,-21.0,17.0
2,3,1400,60,20,0.153846,1,-0.059639,0.269418,1.444542,2.940728,...,106.107778,-1085.918303,292.417343,171.931801,215.611943,-102.875918,9773.418207,10.0,35.0,12.0
3,4,1400,70,10,0.153846,1,-0.051020,0.211907,1.601667,3.661974,...,44.003239,337.234321,311.972632,198.723230,176.177248,15.599799,3261.639986,10.0,-10.0,20.0
4,5,1400,70,15,0.153846,1,-0.100744,0.179582,1.095031,-0.016799,...,99.227805,114.882214,601.144018,368.952328,557.228141,177.572674,13486.874415,6.0,41.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,50,1600,70,15,0.846154,7,-0.084714,0.284958,1.905742,8.950274,...,70.152597,87.522039,823.146921,429.867587,1175.677841,133.585942,17872.244177,4.0,-23.0,2.0
374,51,1600,70,20,0.846154,7,-0.094956,0.257101,1.669120,9.627879,...,68.187862,1113.599765,979.362101,773.237350,771.845687,75.359542,20202.326649,0.0,-1.0,2.0
375,52,1600,80,10,0.846154,7,-0.203323,0.173404,1.671576,4.696642,...,66.605425,1338.936521,663.567303,623.167416,430.622244,4.450439,22266.935055,-2.0,-41.0,5.0
376,53,1600,80,15,0.846154,7,-0.099644,0.266207,1.686495,10.497974,...,51.738010,532.282952,499.039605,280.783456,645.597969,125.019671,9328.366889,10.0,-90.0,1.0


In [21]:
colsY = [c for c in df.columns if re.search(r"stress_value", c)]
print(colsY)
colsX = [c for c in df.columns if c not in colsY]
print(colsX)

['stress_value_5052', 'stress_value_6061', 'stress_value_center']
['sample_no', 'R', 'W', 'D', 'position', 'location', 'Fx_location', 'Fy_location', 'Fz_location', 'Mz_location', 'loc_idx', 'loc_time', 'Fx__dwell__fft_coefficient__attr_"abs"__coeff_11', 'Fx__dwell__ar_coefficient__coeff_0__k_10', 'Fx__dwell__variation_coefficient', 'Fx__dwell__fft_coefficient__attr_"abs"__coeff_51', 'Fx__dwell__fft_coefficient__attr_"angle"__coeff_75', 'Fx__dwell__quantile__q_0.7', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_62', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_42', 'Fx__dwell__quantile__q_0.8', 'Fx__dwell__fft_coefficient__attr_"angle"__coeff_62', 'Fx__dwell__partial_autocorrelation__lag_6', 'Fx__dwell__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.2', 'Fx__dwell__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.2', 'Fx__dwell__fft_coefficient__attr_"real"__coeff_31', 'Fx__dwell__fft_coefficient__attr_"angle"__coeff_37', 'Fy__dwell__fft_coefficient__attr_"real"__

In [22]:
dfY = df[colsY]
dfX_all = df[colsX]

In [23]:
if MAKE_ONE_HOT:
    dft1 = pd.get_dummies(
        dfX_all, columns=["location", "R", "W", "D"], drop_first=False, dtype=int
    )
    dfX = dft1.drop(columns=["sample_no"])
else:
    dfX = dfX_all.drop(columns=["sample_no", "location"])
dfX

,R,W,D,position,Fx_location,Fy_location,Fz_location,Mz_location,loc_idx,loc_time,...,"Mz__weld__fft_coefficient__attr_""angle""__coeff_15",Mz__weld__last_location_of_maximum,Mz__weld__first_location_of_maximum,"Mz__weld__fft_coefficient__attr_""angle""__coeff_16","Mz__weld__fft_coefficient__attr_""real""__coeff_15","Mz__weld__fft_coefficient__attr_""abs""__coeff_58","Mz__weld__fft_coefficient__attr_""abs""__coeff_83","Mz__weld__fft_coefficient__attr_""abs""__coeff_96","Mz__weld__fft_coefficient__attr_""angle""__coeff_37","Mz__weld__fft_coefficient__attr_""imag""__coeff_2"
0,1400,60,10,0.153846,-0.077671,0.143026,1.244326,1.873865,5856,58.560000,...,93.404989,0.034228,0.034151,48.966568,-50.258868,770.788243,599.048302,791.064441,-15.139956,6594.092485
1,1400,60,15,0.153846,-0.133276,0.164254,1.203367,-1.054677,6262,62.620000,...,40.096726,0.010845,0.010768,-4.784945,1601.554004,358.604392,111.493021,204.891839,-83.550988,6665.135623
2,1400,60,20,0.153846,-0.059639,0.269418,1.444542,2.940728,6824,68.240000,...,149.473672,0.046766,0.046689,106.107778,-1085.918303,292.417343,171.931801,215.611943,-102.875918,9773.418207
3,1400,70,10,0.153846,-0.051020,0.211907,1.601667,3.661974,5378,53.782857,...,75.561565,0.055815,0.055725,44.003239,337.234321,311.972632,198.723230,176.177248,15.599799,3261.639986
4,1400,70,15,0.153846,-0.100744,0.179582,1.095031,-0.016799,5959,59.592857,...,85.348076,0.000179,0.000090,99.227805,114.882214,601.144018,368.952328,557.228141,177.572674,13486.874415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,1600,70,15,0.846154,-0.084714,0.284958,1.905742,8.950274,13757,137.565714,...,87.390014,0.989411,0.989322,70.152597,87.522039,823.146921,429.867587,1175.677841,133.585942,17872.244177
374,1600,70,20,0.846154,-0.094956,0.257101,1.669120,9.627879,14329,143.285714,...,65.122289,0.975592,0.975503,68.187862,1113.599765,979.362101,773.237350,771.845687,75.359542,20202.326649
375,1600,80,10,0.846154,-0.203323,0.173404,1.671576,4.696642,12049,120.490000,...,58.113275,0.005230,0.005128,66.605425,1338.936521,663.567303,623.167416,430.622244,4.450439,22266.935055
376,1600,80,15,0.846154,-0.099644,0.266207,1.686495,10.497974,12631,126.310000,...,68.126310,0.934468,0.934366,51.738010,532.282952,499.039605,280.783456,645.597969,125.019671,9328.366889


In [24]:
# Extract features and targets
_X = dfX.values
_Y = dfY.values

print(_X.shape)
print(_Y.shape)

(378, 130)
(378, 3)


In [25]:
dfY = df[colsY]
dfX_all = df[colsX]

In [26]:
_X_train, _X_test, _Y_train, _Y_test = train_test_split(
    _X, _Y, test_size=0.3, random_state=0
)
print(_X_train.shape)
print(_X_test.shape)
print(_Y_train.shape)
print(_Y_test.shape)

(264, 130)
(114, 130)
(264, 3)
(114, 3)


In [27]:
scX = StandardScaler()
X_train = scX.fit_transform(_X_train)
X_test = scX.transform(_X_test)

scY = StandardScaler()
Y_train = scY.fit_transform(_Y_train)
Y_test = scY.transform(_Y_test)

In [28]:
paramSet = {
    "ex1": {"n_estimators": 10, "learning_rate": 1.0, "max_depth": 2},
    "ex2": {"n_estimators": 50, "learning_rate": 1.0, "max_depth": 3},
    "ex3": {"n_estimators": 50, "learning_rate": 1.0, "max_depth": 10},
    "ex4": {"n_estimators": 50, "learning_rate": 1.0, "max_depth": 1},
    "ex5": {"n_estimators": 100, "learning_rate": 1.0, "max_depth": 1},
    "ex6": {"n_estimators": 200, "learning_rate": 1.0, "max_depth": 1},
}

params = paramSet["ex4"]

In [29]:
base = GradientBoostingRegressor(**params)
regressor = MultiOutputRegressor(base)

In [30]:
regressor.fit(X_train, Y_train)
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)

In [31]:
dfFi = pd.DataFrame(data={"feature": dfX.columns.values})
estimators = regressor.estimators_
for i in range(len(estimators)):
    fi = estimators[i].feature_importances_
    dfFi[f"importance_{i + 1}"] = fi

dfFi

,feature,importance_1,importance_2,importance_3
0,R,0.000000,0.000000,0.000000
1,W,0.000000,0.000000,0.000000
2,D,0.003327,0.000000,0.000000
3,position,0.004486,0.036157,0.429059
4,Fx_location,0.031788,0.059419,0.064723
...,...,...,...,...
125,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58",0.000000,0.000000,0.000000
126,"Mz__weld__fft_coefficient__attr_""abs""__coeff_83",0.000000,0.000000,0.014861
127,"Mz__weld__fft_coefficient__attr_""abs""__coeff_96",0.076130,0.000000,0.000000
128,"Mz__weld__fft_coefficient__attr_""angle""__coeff_37",0.000000,0.000000,0.000000


In [32]:
display(dfFi.sort_values(by="importance_1", ascending=False))

,feature,importance_1,importance_2,importance_3
25,"Fy__dwell__fft_coefficient__attr_""real""__coeff_71",0.255314,0.000000,0.000000
40,"Fz__dwell__fft_coefficient__attr_""angle""__coeff_4",0.153842,0.000000,0.000000
8,loc_idx,0.093112,0.073810,0.000000
127,"Mz__weld__fft_coefficient__attr_""abs""__coeff_96",0.076130,0.000000,0.000000
6,Fz_location,0.070330,0.110061,0.026085
...,...,...,...,...
122,Mz__weld__first_location_of_maximum,0.000000,0.000000,0.000000
125,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58",0.000000,0.000000,0.000000
126,"Mz__weld__fft_coefficient__attr_""abs""__coeff_83",0.000000,0.000000,0.014861
128,"Mz__weld__fft_coefficient__attr_""angle""__coeff_37",0.000000,0.000000,0.000000


In [33]:
display(dfFi.sort_values(by="importance_2", ascending=False))

,feature,importance_1,importance_2,importance_3
76,"Fx__weld__fft_coefficient__attr_""angle""__coeff_31",0.000000,0.223421,0.000000
121,Mz__weld__last_location_of_maximum,0.000000,0.110458,0.000000
6,Fz_location,0.070330,0.110061,0.026085
8,loc_idx,0.093112,0.073810,0.000000
33,"Fy__dwell__fft_coefficient__attr_""angle""__coef...",0.000000,0.071456,0.000000
...,...,...,...,...
125,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58",0.000000,0.000000,0.000000
126,"Mz__weld__fft_coefficient__attr_""abs""__coeff_83",0.000000,0.000000,0.014861
127,"Mz__weld__fft_coefficient__attr_""abs""__coeff_96",0.076130,0.000000,0.000000
128,"Mz__weld__fft_coefficient__attr_""angle""__coeff_37",0.000000,0.000000,0.000000


In [34]:
display(dfFi.sort_values(by="importance_3", ascending=False))

,feature,importance_1,importance_2,importance_3
3,position,0.004486,0.036157,0.429059
99,Fy__weld__ar_coefficient__coeff_6__k_10,0.000000,0.000000,0.124249
110,Fz__weld__ratio_beyond_r_sigma__r_1,0.000000,0.000000,0.069014
5,Fy_location,0.026650,0.058117,0.065646
4,Fx_location,0.031788,0.059419,0.064723
...,...,...,...,...
123,"Mz__weld__fft_coefficient__attr_""angle""__coeff_16",0.000000,0.000000,0.000000
125,"Mz__weld__fft_coefficient__attr_""abs""__coeff_58",0.000000,0.000000,0.000000
127,"Mz__weld__fft_coefficient__attr_""abs""__coeff_96",0.076130,0.000000,0.000000
128,"Mz__weld__fft_coefficient__attr_""angle""__coeff_37",0.000000,0.000000,0.000000
